<a href="https://colab.research.google.com/github/sandy0216/debate/blob/main/%E6%8E%92%E8%B3%BD%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
import csv
from ortools.graph import pywrapgraph
import numpy as np

In [ ]:
filename = '/content/drive/MyDrive/63屆寒菁/coding_test/schools.csv'
schools = []
with open(filename, newline='') as csvfile:
  rows = csv.reader(csvfile)
  for row in rows:
    schools.append(row)

In [ ]:
class battle: #每個循環的資訊
  def __init__(self,cycle):
    self.sch  = []
    self.cycle = cycle
    self.time = []
    self.ind = 0
  
  def check_time(self):
    if(len(self.time)!=0):
      self.time = []

class scheule: #每個時段的資訊，希望可以擴充到排裁判
  def __init__(self,time):
    self.time = time
    self.battle = []
    self.label = None
    self.room = None
  
  def check_battle(self):
    if(len(self.battle)!=0):
      self.battle = []

In [ ]:
#將csv檔中的學校資訊存成battle資料格式
def create_battle(schools):
  battles = []
  n = len(schools)
  for i in range(n):
    school = schools[i]
    n_sch  = len(school)
    temp = battle(school[0])
    for i in range(1,n_sch):
      temp.sch.append(school[i])
    battles.append(temp)
  return battles
battles = create_battle(schools)

In [ ]:
#將時段/場次資訊存成scheule資料格式
times = {'3/4(五)時段一 10.00-12.30':7,'3/4(五)時段二 13.00-15.30':7\
    ,'3/5(六)時段一 9.00-11.30':5,'3/5(六)時段二 12.00-14.30':5}
def create_scheule(times):
  scheules = []
  for i,t in enumerate(times.keys()):
    temp = scheule(i)
    temp.label = t
    temp.room = times[t]
    scheules.append(temp)
  return scheules
scheules = create_scheule(times)
print([t.room for t in scheules])

[7, 7, 5, 5]


In [ ]:
cycle_cap = np.array([len(bat.sch) for bat in battles]) # 每個循環有幾隊
time_cap = np.array([t.room for t in scheules])     # 每個時段有幾場

n_cycle, n_time = len(cycle_cap), len(time_cap)
vertex = np.arange(0,n_cycle+n_time+2,1)
st, end = int(vertex[0]),int(vertex[-1])
cycle, time = vertex[1:n_cycle+1],vertex[n_cycle+1:-1]
start_nodes = []
end_nodes = []
capacities = []
def add_edge(st,end,cap):
  start_nodes.append(int(st))
  end_nodes.append(int(end))
  capacities.append(int(cap))
for i in range(n_cycle):
  add_edge(st,cycle[i],cycle_cap[i])
for i in range(n_time):
  add_edge(time[i],end,time_cap[i])
for i in range(n_cycle):
  for j in range(n_time):
    add_edge(cycle[i],time[j],1)

In [ ]:
max_flow = pywrapgraph.SimpleMaxFlow()
# Add each arc.
for i in range(0, len(start_nodes)):
  max_flow.AddArcWithCapacity(start_nodes[i], end_nodes[i], capacities[i])

# Find the maximum flow between node 0 and node 4.
if max_flow.Solve(st, end) == max_flow.OPTIMAL:
  print('[Info] Get the optimal flow!')
if(max_flow.OptimalFlow()!=np.sum(cycle_cap)):
  print('[Error] Fail to solve the flow network')

for bat in battles:
  bat.check_time()
for sch in scheules:
  sch.check_battle()

for i in range(max_flow.NumArcs()):
  if(max_flow.Flow(i)==1):
    #print(max_flow.Tail(i)-1,max_flow.Head(i)-time[0])
    battles[max_flow.Tail(i)-1].time.append(max_flow.Head(i)-time[0])
    scheules[max_flow.Head(i)-time[0]].battle.append(max_flow.Tail(i)-1)

[Info] Get the optimal flow!


In [ ]:
for sch in scheules:
  print(sch.label)
  cyc_arr = sch.battle
  for cyc in cyc_arr:
    cyc_sch = battles[cyc].sch
    cyc_ind = battles[cyc].ind
    print(f'{cyc_sch[cyc_ind%3]}v.s.{cyc_sch[(cyc_ind+1)%3]}',end='\t')
    battles[cyc].ind += 1
  print('')

3/4(五)時段一 10.00-12.30
海洋大學v.s.政大演辯A	北科正言Bv.s.成功大學B	輔仁大學v.s.臺灣大學B	清華大學v.s.中山大學	東吳正言v.s.成功大學A	文化大學v.s.北科正言A	國北教大v.s.臺灣大學A	
3/4(五)時段二 13.00-15.30
政大演辯Av.s.北大滔滔A	成功大學Bv.s.世新大學B	臺灣大學Bv.s.中正大學	世新大學Av.s.政大演辯B	成功大學Av.s.北大滔滔B	北科正言Av.s.淡江大學	臺灣大學Av.s.政大法辯	
3/5(六)時段一 9.00-11.30
中山大學v.s.中央大學	政大演辯Bv.s.東海大學	北大滔滔Bv.s.東吳正言	淡江大學v.s.文化大學	政大法辯v.s.國北教大	
3/5(六)時段二 12.00-14.30
北大滔滔Av.s.海洋大學	世新大學Bv.s.北科正言B	中正大學v.s.輔仁大學	中央大學v.s.清華大學	東海大學v.s.世新大學A	


In [ ]:
f = open('/content/drive/MyDrive/63屆寒菁/coding_test/schedule.csv','w')
for sch in scheules:
  print(sch.label,file=f)
  cyc_arr = sch.battle
  for cyc in cyc_arr:
    cyc_sch = battles[cyc].sch
    cyc_ind = battles[cyc].ind
    print(f'{cyc_sch[cyc_ind%3]}x{cyc_sch[(cyc_ind+1)%3]}',end=',',file=f)
    battles[cyc].ind += 1
  print('',file=f)
  for cyc in cyc_arr:
    cyc_sch = battles[cyc].sch
    cyc_ind = battles[cyc].ind
    print(f'{cyc_sch[(cyc_ind+1)%3]}',end=',',file=f)
    battles[cyc].ind += 1
  print('',file=f)
f.close()